#This is an example of performing Video Activity Recognition using LSTM
Modified from "Hands-on Computer Vision with TensorFlow 2" by B. Planche and E. Andres

In [ ]:
# STEP 1:  Install packages in the current environment
import sys
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install tqdm
!{sys.executable} -m pip install scikit-learn

In [2]:
#!{sys.executable} -m pip install tensorflow==1.14
!{sys.executable} -m pip install tensorflow==2.0

#Step 2: import modules

In [3]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import LabelBinarizer


In [ ]:
#tf.enable_eager_execution()
#tf.executing_eagerly()

#Step 3: setup variables

In [ ]:
#location of where YOU have installed the data set UCF-101 located
# at 
#BASE_PATH = '../data/UCF-101'
#change the base path to location YOU installed UCF-101 dataset
#BASE_PATH = 'C:/Grewe/Classes/CS663/Mat/LSTM/data/UCF-101'
BASE_PATH = 'C:\\Users\\STSC.LNVO-126908.000\\Desktop\\FallDetection\\Classes\\'
VIDEOS_PATH = os.path.join(BASE_PATH, '**','*.mp4')

#this specifies the sequence length will process by LSTM
SEQUENCE_LENGTH = 40
BATCH_SIZE = 16
print(VIDEOS_PATH)

### STEP 4:  sample the video --do not process every frame
PART 1: define function frame_generator() that creates Sequence_length samples by taking every Kth sample were K= num_frames_in_video / SEQUENCE LENGTH     PART 2: you load the DataSet and specify the output will be frames of size 299x299 x3(rgb) AND you create batches of 16 together at a time

In [ ]:
def frame_generator():
    video_paths = tf.io.gfile.glob(VIDEOS_PATH)
    np.random.shuffle(video_paths)
    for video_path in video_paths:
        frames = []
        cap = cv2.VideoCapture(video_path)
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        sample_every_frame = max(1, num_frames // SEQUENCE_LENGTH)
        current_frame = 0
        
        label = os.path.basename(os.path.dirname(video_path))
        
        max_images = SEQUENCE_LENGTH
        while True:
            success, frame = cap.read()
            if not success:
                break
                
            if current_frame % sample_every_frame == 0:
                frame = frame[:, :, ::-1]
                img = tf.image.resize(frame, (224, 224))
                img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
                max_images -= 1
                yield img, video_path
                
            if max_images == 0:
                break
            current_frame += 1
            

In [ ]:
dataset = tf.data.Dataset.from_generator(frame_generator,
                                         output_types=(tf.float32, tf.string),
                                         output_shapes=((224, 224, 3), ()))

dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
print(dataset)

### STEP 5: 
For Feature Extraction we are going to use a existing CNN model called Inception V3 which is built into TensorFlow

In [ ]:
mobilenet_v2 = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
x = mobilenet_v2.output

# We add Average Pooling to transform the feature map from
# 8 * 8 * 2048 to 1 x 2048, as we don't need spatial information
pooling_output = tf.keras.layers.GlobalAveragePooling2D()(x)
feature_extraction_model = tf.keras.Model(mobilenet_v2.input,pooling_output)


### STEP 6: 
Extract Features using our InceptionV3 CNN model

In [ ]:
#Don't run this for if done with feature extraction 
current_path = None
all_features = []

#cycle through the dataset and visit each image, note the tdqm is a progress bar
#that updates each time a new iteration is called 
#call feature_extraction_model above (Inception v3) for the image to extract the features
for img, batch_paths in tqdm.tqdm(dataset):
    batch_features = feature_extraction_model(img)
    #reshape the tensor 
    batch_features = tf.reshape(batch_features, 
                              (batch_features.shape[0], -1))
    
    for features, path in zip(batch_features.numpy(), batch_paths.numpy()):
        if path != current_path and current_path is not None:
            output_path = current_path.decode().replace('.mp4', '.npy')
            np.save(output_path, all_features)
            all_features = []
            
        current_path = path
        all_features.append(features)

### STEP 7: 
Create a MyLabelBinarizer for 2 Classes

In [ ]:
#run this

class MyLabelBinarizer(LabelBinarizer):
    def transform(self, y):
        Y = super().transform(y)
        if self.y_type_ == 'binary':
            return np.hstack((Y, 1-Y))
        else:
            return Y
    def inverse_transform(self, Y, threshold=None):
        if self.y_type_ == 'binary':
            return super().inverse_transform(Y[:, 0], threshold)
        else:
            return super().inverse_transform(Y, threshold)

In [ ]:
#run this
LABELS = ['Falling','Walking']
encoder = MyLabelBinarizer()
encoder.fit(LABELS)
print(encoder.classes_)
print(encoder.transform(['Falling', 'Walking']))

t= encoder.transform(['Falling', 'Walking', 'Walking'])
print(t)
print(encoder.inverse_transform(t))
print("length of labrels " + str(len(LABELS)))

### STEP 8: 
    Create the LSTM model:    1) Masking layer  2) LSTM layer with 512 cells, dropout 0.5, recurrent_dropout of 0.5  
 3) a fully connected relu activation layer with 256 outputs,  4) a droupout layer 0.5  5) a final decision fully connected layer of putput length of labels  (which is the number of classes) with softmax activation.

In [ ]:
#setup a keras Sequential model with 1) Masking layer  2) LSTM layer with 512 cells, dropout 0.5, recurrent_dropout of 0.5  
# 3) a fully connected relu activation layer with 256 outputs,  4) a droupout layer 5) a final decision fully connected layer of length of labels
# (which is the number of classes) with softmax activation.
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0.),
    tf.keras.layers.LSTM(512, dropout=0.5, recurrent_dropout=0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
    #tf.keras.layers.Dense(len(LABELS), activation='softmax')
])

### STEP 8: 
Setup for the model the Loss function, the Optimizer function, and any metrics want to compute in training

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

### STEP 9: 
Setup  the training and test list which are lists of the training filenames.   Note you will need to change the location of these files to point to your location.  Define a function make_generator that returns a generator which will randomly shuffle a file list (either training or testing that will be passed later) and then changes the file extension of the avi files listed in the list to .npy which is our features for that avi video which were calcluated in step 6

In [ ]:
#train_file = '/Users/subhi/Downloads/CV-proj3/Data/trainlist.txt'
#test_file = '/Users/subhi/Downloads/CV-proj3/Data/testlist.txt'
test_file = 'C:/Users/STSC.LNVO-126908.000/Desktop/FallDetection/trainlist.txt'
train_file = 'C:/Users/STSC.LNVO-126908.000/Desktop/FallDetection/testlist.txt'

with open(test_file) as f:
    test_list = [row.strip() for row in list(f)]

with open(train_file) as f:
     train_list = [row.strip() for row in list(f)]
    #train_list = [row.split(' ')[0] for row in train_list]
#print(train_list)
def make_generator(file_list):
    def generator():
        np.random.shuffle(file_list)
        for path in file_list:
            full_path = os.path.join(BASE_PATH, path).replace('.mp4', '.npy')
            
            label = os.path.basename(os.path.dirname(path))
            features = np.load(full_path)
            
            
            padded_sequence = np.zeros((SEQUENCE_LENGTH, 1280))
            padded_sequence[0:len(features)] = np.array(features)
            
            transformed_label = encoder.transform([label])
            
            yield padded_sequence, transformed_label[0]
    return generator

In [ ]:
print(train_list)

In [ ]:
print(test_list)

### STEP 10: 
Setup the train_dataset and valid_dataset (validation/testing).   Here we setting up training batch sets of 16.  

In [ ]:
#for tesnorflow 2.*
train_dataset = tf.data.Dataset.from_generator(make_generator(train_list),
                output_types=(tf.float32, tf.int16),
                output_shapes=((SEQUENCE_LENGTH, 1280), (len(LABELS))))
                 

train_dataset = train_dataset.batch(16,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_generator(make_generator(test_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=((SEQUENCE_LENGTH, 1280), (len(LABELS))))
valid_dataset = valid_dataset.batch(16,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
#for tensorflow 1.1.4
train_dataset = tf.data.Dataset.from_generator(make_generator(train_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=(tf.TensorShape([SEQUENCE_LENGTH, 1280]), tf.TensorShape([len(LABELS)])))

train_dataset = train_dataset.batch(16,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_generator(make_generator(test_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=(tf.TensorShape([SEQUENCE_LENGTH, 1280]), tf.TensorShape([len(LABELS)])))
valid_dataset = valid_dataset.batch(16,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
print(train_dataset)

In [ ]:
print(valid_dataset)

In [ ]:
BASE_DATA_PATH = 'C:/Users/STSC.LNVO-126908.000/Desktop/FallDetection'
mylog_dir = os.path.join( BASE_DATA_PATH, "train_log")
print("Mylog directory = " + mylog_dir)

In [ ]:
#tf 2.0
tensorboard_callback = tf.keras.callbacks.TensorBoard(os.path.join('tmp'), update_freq=1000)
#model.fit(train_dataset, epochs=1, callbacks=[tensorboard_callback], validation_data=valid_dataset)
model.fit(train_dataset, epochs=1,validation_data=valid_dataset)

In [ ]:
model.summary()

In [ ]:
#tf 1.1.4
model.fit(train_dataset,epochs=1, validation_data= valid_dataset, validation_steps=4,verbose=0)

In [ ]:
print(tf.__version__)

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=mylog_dir, update_freq=1000)
print(os.path.join(mylog_dir, 'train'))
#following call works for tensorflow 1.1.4
model.fit(train_dataset, 
          epochs=17, 
          validation_data=valid_dataset, 
          callbacks=[tensorboard_callback])

#model.fit(train_dataset, epochs=17, callbacks=[tensorboard_callback], validation_data=valid_dataset, validation_steps=4, verbose=0)

In [ ]:
model.file=os.path.join(BASE_PATH,'my_model.h5')
model.save(model.file)

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])


### STEP 12:  save the tensorflow model to an h5 file

In [ ]:
BASE_DATA_PATH = '/Users/subhi/Downloads/CV-Proj3/Data'
model_file = os.path.join(BASE_DATA_PATH, 'my_model.h5')
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model shuold be saved to HDF5.
model.save(model_file) 

### STEP 13: try to conver the model to tflite --- Support to come 2019 (when?)--Curently LSTM conversion to TFLite NOT supported

In [ ]:
#from tensorflow import lite
tflite_file  = os.path.join(BASE_DATA_PATH, 'my_tflite_model.tflite')
print(" want to save tflite_file" + tflite_file)
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()




#now save the tflite model to the file
#tflite_model.save(tflite_file)   #Note this does not seem to work although in google documentation
open(tflite_file, "wb").write(tflite_model)

### STEP 14: run evaluation on the test data feature extraction

In [ ]:
# evaluate the test data using model


# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')

# NOTE: should have separate test data but, only have validation data
#results = model.evaluate_generator(val_data_gen, verbose=1)
results = model.evaluate(valid_dataset, verbose=1)
print('test loss, test acc:', results)




### STEP 15: Run predictions on the test data feature extracted

In [ ]:
# make predictions

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions ')
predictions = model.predict(valid_dataset, verbose=1 )


In [ ]:
#print out prediction info for validation data set (as do not have separate test data set)
print('predictions shape:', predictions.shape)
print(predictions)
print(len(predictions))
